# L6: Multi-agent Collaboration for Financial Analysis

In this lesson, you will learn ways for making agents collaborate with each other.

The libraries are already installed in the classroom. If you're running this notebook on your own machine, you can install the following:
```Python
!pip install crewai==0.28.8 crewai_tools==0.1.6 langchain_community==0.0.29
```

In [1]:
# Warning control
import warnings
warnings.filterwarnings('ignore')

- Import libraries, APIs and LLM

In [2]:
from crewai import Agent, Task, Crew

**Note**: 
- The video uses `gpt-4-turbo`, but due to certain constraints, and in order to offer this course for free to everyone, the code you'll run here will use `gpt-3.5-turbo`.
- You can use `gpt-4-turbo` when you run the notebook _locally_ (using `gpt-4-turbo` will not work on the platform)
- Thank you for your understanding!

In [3]:
# from utils import get_openai_api_key, get_serper_api_key
# openai_api_key = get_openai_api_key()
# os.environ["OPENAI_MODEL_NAME"] = 'gpt-3.5-turbo'
# os.environ["SERPER_API_KEY"] = get_serper_api_key()

import os
from langchain_openai import ChatOpenAI

os.environ["OPENAI_API_KEY"] = "sk-proj-1111" # dummy key
llm = ChatOpenAI(
    model="ollama/mistral:latest",
    base_url="http://localhost:11434/v1"
)
SERPER_API_KEY = '229e632bb96cefd7d7f57c28a71aa35747674c9d'

## crewAI Tools

In [4]:
from crewai_tools import ScrapeWebsiteTool, SerperDevTool
search_tool = SerperDevTool()
scrape_tool = ScrapeWebsiteTool()

## Creating Agents

In [5]:
data_analyst_agent = Agent(
    role="Data Analyst",
    goal="Monitor and analyze market data in real-time "
         "to identify trends and predict market movements.",
    backstory="Specializing in financial markets, this agent "
              "uses statistical modeling and machine learning "
              "to provide crucial insights. With a knack for data, "
              "the Data Analyst Agent is the cornerstone for "
              "informing trading decisions.",
    verbose=True,
    llm=llm,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [6]:
trading_strategy_agent = Agent(
    role="Trading Strategy Developer",
    goal="Develop and test various trading strategies based "
         "on insights from the Data Analyst Agent.",
    backstory="Equipped with a deep understanding of financial "
              "markets and quantitative analysis, this agent "
              "devises and refines trading strategies. It evaluates "
              "the performance of different approaches to determine "
              "the most profitable and risk-averse options.",
    verbose=True,
    llm=llm,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [7]:
execution_agent = Agent(
    role="Trade Advisor",
    goal="Suggest optimal trade execution strategies "
         "based on approved trading strategies.",
    backstory="This agent specializes in analyzing the timing, price, "
              "and logistical details of potential trades. By evaluating "
              "these factors, it provides well-founded suggestions for "
              "when and how trades should be executed to maximize "
              "efficiency and adherence to strategy.",
    verbose=True,
    llm=llm,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

In [8]:
risk_management_agent = Agent(
    role="Risk Advisor",
    goal="Evaluate and provide insights on the risks "
         "associated with potential trading activities.",
    backstory="Armed with a deep understanding of risk assessment models "
              "and market dynamics, this agent scrutinizes the potential "
              "risks of proposed trades. It offers a detailed analysis of "
              "risk exposure and suggests safeguards to ensure that "
              "trading activities align with the firm’s risk tolerance.",
    verbose=True,
    llm=llm,
    allow_delegation=True,
    tools = [scrape_tool, search_tool]
)

## Creating Tasks

In [9]:
# Task for Data Analyst Agent: Analyze Market Data
data_analysis_task = Task(
    description=(
        "Continuously monitor and analyze market data for "
        "the selected stock ({stock_selection}). "
        "Use statistical modeling and machine learning to "
        "identify trends and predict market movements."
    ),
    expected_output=(
        "Insights and alerts about significant market "
        "opportunities or threats for {stock_selection}."
    ),
    agent=data_analyst_agent,
)

In [10]:
# Task for Trading Strategy Agent: Develop Trading Strategies
strategy_development_task = Task(
    description=(
        "Develop and refine trading strategies based on "
        "the insights from the Data Analyst and "
        "user-defined risk tolerance ({risk_tolerance}). "
        "Consider trading preferences ({trading_strategy_preference})."
    ),
    expected_output=(
        "A set of potential trading strategies for {stock_selection} "
        "that align with the user's risk tolerance."
    ),
    agent=trading_strategy_agent,
)


In [11]:
# Task for Trade Advisor Agent: Plan Trade Execution
execution_planning_task = Task(
    description=(
        "Analyze approved trading strategies to determine the "
        "best execution methods for {stock_selection}, "
        "considering current market conditions and optimal pricing."
    ),
    expected_output=(
        "Detailed execution plans suggesting how and when to "
        "execute trades for {stock_selection}."
    ),
    agent=execution_agent,
)


In [12]:
# Task for Risk Advisor Agent: Assess Trading Risks
risk_assessment_task = Task(
    description=(
        "Evaluate the risks associated with the proposed trading "
        "strategies and execution plans for {stock_selection}. "
        "Provide a detailed analysis of potential risks "
        "and suggest mitigation strategies."
    ),
    expected_output=(
        "A comprehensive risk analysis report detailing potential "
        "risks and mitigation recommendations for {stock_selection}."
    ),
    agent=risk_management_agent,
)

## Creating the Crew
- The `Process` class helps to delegate the workflow to the Agents (kind of like a Manager at work)
- In the example below, it will run this hierarchically.
- `manager_llm` lets you choose the "manager" LLM you want to use.

In [13]:
from crewai import Crew, Process
from langchain_openai import ChatOpenAI

# Define the crew with agents and tasks
financial_trading_crew = Crew(
    agents=[data_analyst_agent, 
            trading_strategy_agent, 
            execution_agent, 
            risk_management_agent],
    
    tasks=[data_analysis_task, 
           strategy_development_task, 
           execution_planning_task, 
           risk_assessment_task],
    
    # manager_llm=ChatOpenAI(model="gpt-3.5-turbo", temperature=0.7),
    manager_llm = ChatOpenAI(model="ollama/mistral:latest", base_url="http://localhost:11434/v1", temperature=0.4),
    process=Process.hierarchical,
    verbose=True
)


## Running the Crew

- Set the inputs for the execution of the crew.

In [14]:
# Example data for kicking off the process
financial_trading_inputs = {
    'stock_selection': 'AAPL',
    'initial_capital': '100000',
    'risk_tolerance': 'Medium',
    'trading_strategy_preference': 'Day Trading',
    'news_impact_consideration': True
}

**Note**: LLMs can provide different outputs for they same input, so what you get might be different than what you see in the video.

In [15]:
### this execution will take some time to run
result = financial_trading_crew.kickoff(inputs=financial_trading_inputs)

# Agent: Crew Manager
## Task: Continuously monitor and analyze market data for the selected stock (AAPL). Use statistical modeling and machine learning to identify trends and predict market movements.
 Error parsing LLM output, agent will retry: I did it wrong. Tried to both perform Action and give a Final Answer at the same time, I must do one or the other
 Error parsing LLM output, agent will retry: I did it wrong. Tried to both perform Action and give a Final Answer at the same time, I must do one or the other
 Error parsing LLM output, agent will retry: I did it wrong. Tried to both perform Action and give a Final Answer at the same time, I must do one or the other
 Error parsing LLM output, agent will retry: I did it wrong. Tried to both perform Action and give a Final Answer at the same time, I must do one or the other
 Error parsing LLM output, agent will retry: I did it wrong. Tried to both perform Action and give a Final Answer at the same time, I must do one or the other
 Er

Final Answer: 
Apple Inc. (AAPL) Stock Price, News, Quote & History - Yahoo Finance News Today's news US Politics World Tech Reviews and deals Audio Computing Gaming Health Home Phones Science TVs Climate change Health Science Originals The 360 Newsletters Life Health COVID-19 Fall allergies Health news Mental health Relax Sexual health Studies The Unwind Parenting Family health So mini ways Style and beauty It Figures Unapologetically Horoscopes Shopping Buying guides Food Travel Autos Gift ideas Buying guides Entertainment Celebrity TV Movies Music How to Watch Interviews Videos Finance My Portfolio News Latest Stock Market Originals The Morning Brief Economies Premium News Earnings Tech Housing Crypto Biden Economy Markets Stocks: Most Actives Stocks: Gainers Stocks: Losers Trending Tickers Futures World Indices US Treasury Bonds Rates Currencies Crypto Top ETFs Top Mutual Funds Options: Highest Open Interest Options: Highest Implied Volatility Sectors Basic Materials Communication 

Final Answer: 
Apple Inc. (AAPL) designs, manufactures, and markets smartphones, personal computers, tablets, wearables, and accessories worldwide. The company offers iPhone, Mac, iPad, and wearables, along with various subscription-based services like Apple Arcade, Apple Fitness+, Apple Music, Apple News+, Apple TV+, and Apple Card. They also operate platforms like the App Store and provide cloud services. AAPL's market cap is 3.676T, with a profit margin of 23.97% and a return on equity of 157.41%. Trailing total returns as of 1/3/2025 show a YTD return of 3.58% and a 1-year return of 31.64%, outperforming the S&P 500. AAPL's stock price is currently 243.18, with a 52-week range of 164.08 - 260.10. As of 12:04:32 PM EST, AAPL is down 0.27%. The company has 164,000 full-time employees and is headquartered in Cupertino, California.

> Finished chain.
 [DEBUG]: [Crew Manager] Task output: Apple Inc. (AAPL) designs, manufactures, and markets smartphones, personal computers, tablets, wear

Final Answer: 
Apple Inc. (AAPL) Stock Price, News, Quote & History - Yahoo Finance News Today's news US Politics World Tech Reviews and deals Audio Computing Gaming Health Home Phones Science TVs Climate change Health Science Originals The 360 Newsletters Life Health COVID-19 Fall allergies Health news Mental health Relax Sexual health Studies The Unwind Parenting Family health So mini ways Style and beauty It Figures Unapologetically Horoscopes Shopping Buying guides Food Travel Autos Gift ideas Buying guides Entertainment Celebrity TV Movies Music How to Watch Interviews Videos Finance My Portfolio News Latest Stock Market Originals The Morning Brief Economies Premium News Earnings Tech Housing Crypto Biden Economy Markets Stocks: Most Actives Stocks: Gainers Stocks: Losers Trending Tickers Futures World Indices US Treasury Bonds Rates Currencies Crypto Top ETFs Top Mutual Funds Options: Highest Open Interest Options: Highest Implied Volatility Sectors Basic Materials Communication 

Final Answer: 
After gathering insights from the Data Analyst, I have developed a set of potential trading strategies for AAPL that align with the user's risk tolerance. Based on AAPL's financials, market performance, and recent trends, the strategies focus on maximizing returns while managing risk. These strategies include options trading, long-term investing, and leveraging AAPL's subscription-based services to generate additional revenue streams. With a thorough understanding of AAPL's market cap, profit margin, return on equity, and stock price performance, these strategies aim to capitalize on AAPL's strong position in the market and deliver optimal results for the user.

> Finished chain.
 [DEBUG]: [Crew Manager] Task output: After gathering insights from the Data Analyst, I have developed a set of potential trading strategies for AAPL that align with the user's risk tolerance. Based on AAPL's financials, market performance, and recent trends, the strategies focus on maximizing r

Thought: I have found relevant search results that may contain the approved trading strategies for AAPL.
Action: Read website content
Action Input: {"website_url": "https://marketchameleon.com/Overview/AAPL/StrategyPayout/"} 

Market Chameleon
Access Denied
Possible reasons for receiving this errorCorrective action(s)
Accessing from a virtual machine and/or managed hosting environmentUse a physical, local machine
Using a VPN or other security productDisable it
Using web automation or a non-standard web browser such as a crawlerThis is prohibited under MarketChameleon's Terms of Use
Opening excessive windows and/or tabsClose all MarketChameleon windows/tabs and launch just one
If you feel that you've received this message in error, please email the details that appear below to support@marketchameleon.com
URL: https://marketchameleon.com/Overview/AAPL/StrategyPayout/
Reference Number: 18.d994d817.1735923955.b0d7378f

Thought: I need to find a different source to access the approved tradi

Thought: I need to find a reliable source that provides approved trading strategies for AAPL.
Action: Read website content
Action Input: {"website_url": "https://www.quantifiedstrategies.com/stock-trading-aapl/"} 

I tried reusing the same input, I must stop using this action input. I'll try something else instead.



Thought: I need to find a reliable source that provides approved trading strategies for AAPL.
Action: Read website content
Action Input: {"website_url": "https://www.marketchameleon.com/Overview/AAPL/StrategyPayout/"} 

Market Chameleon
Access Denied
Possible reasons for receiving this errorCorrective action(s)
Accessing from a virtual machine and/or managed hosting environmentUse a physical, local machine
Using a VPN or other security productDisable it
Using web automation or a non-standard web browser such as a crawlerThis is prohibited under MarketChameleon's Terms of Use
Opening excessive windows and/or tabsClose all MarketChameleon windows/tabs and launch just one
If

I need to gather information on the proposed trading strategies and execution plans for AAPL to evaluate the associated risks effectively.

Action: Read website content
Action Input: {"website_url": "https://www.apple.com/investor/"} 

Attention Required! | Cloudflare
Please enable cookies.
Sorry, you have been blocked
You are unable to access web.prd.q4inc.com
Why have I been blocked?
This website is using a security service to protect itself from online attacks. The action you just performed triggered the security solution. There are several actions that could trigger this block including submitting a certain word or phrase, a SQL command or malformed data.
What can I do to resolve this?
You can email the site owner to let them know you were blocked. Please include what you were doing when this page came up and the Cloudflare Ray ID found at the bottom of this page.
Cloudflare Ray ID: 8fc493695e60cf27
•
 Your IP:
 Click to reveal
52.8.83.38
•
Performance & security by Cloudflare

Fin

Thought: I now can give a great answer
Final Answer: Based on the search results, potential risks associated with trading AAPL include a slowdown in overall smartphone market growth, legal and regulatory issues, ability to sell products, production challenges, financial and corporate risks, macroeconomic and political factors, as well as risks related to technology and innovation. To mitigate these risks, traders should consider setting stop-loss orders, diversifying their portfolio, staying informed about market conditions, and implementing real-time risk management systems. Additionally, incorporating risk mitigation strategies while going global and understanding key factors influencing AAPL's market performance are crucial for successful trading execution.

> Finished chain.
 

Based on the search results, potential risks associated with trading AAPL include a slowdown in overall smartphone market growth, legal and regulatory issues, ability to sell products, production challenges,

- Display the final result as Markdown.

In [17]:
from IPython.display import Markdown
Markdown(str(result))

The proposed trading strategies and execution plans for AAPL carry a high risk of market volatility and adverse price movements due to news events or regulatory changes. To minimize these risks, I recommend implementing a diversified portfolio, using hedging strategies such as options or futures, and closely monitoring news events and regulatory changes that could affect AAPL's share price. Additionally, setting stop-loss orders at a wider range can help account for potential market volatility. This report is meant to serve as a guide to minimize risks when trading AAPL shares and should be used in conjunction with professional advice.